# Cycle adatelemzés

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
anomaly_num = 0

In [2]:
df = pd.read_csv("PLC_NBH170.csv", sep=";")

In [3]:
def switch(i):
    switcher = {
        'KBBUD10449-NBH170-PLCStationThing': "10449",
        'KBBUD10704-NBH170-PLCStationThing': "10704",
        'KBBUD10469-NBH170-PLCStationThing': "10469",
        'KBBUD10460-NBH170-PLCStationThing': "10460",
        'KBBUD10395-NBH170-PLCStationThing': "10395",
        'KBBUD10424-NBH170-PLCStationThing': "10424",
        'KBBUD10419-NBH170-PLCStationThing': "10419",
        'KBBUD10396-NBH170-PLCStationThing': "10396"
        
    }
    return switcher.get(i,"invalid")

In [4]:
df['Machine'] = df['Machine'].apply(lambda x: switch(x))
df['StatusTime'] = df['StatusTime'].apply(lambda x: x[:-4])

del df['StatusType']
del df['description']
del df['CreationTime']

# Cycle Státuszok

In [5]:
df = df[df['Status']=='Cycle']
del df['Status']
df = df.sort_values(by='StatusTime')

anomália mentesítés gépenként

In [6]:
list = []
list.append(df[df['Machine']=='10449'])
list.append(df[df['Machine']=='10704'])
list.append(df[df['Machine']=='10469'])
list.append(df[df['Machine']=='10460'])
list.append(df[df['Machine']=='10395'])
list.append(df[df['Machine']=='10424'])
list.append(df[df['Machine']=='10419'])
list.append(df[df['Machine']=='10396'])

In [7]:
# Anomália kezelés 1: több ACTIVE jön egymás után, akkor az ELSŐ érvényes, több END jön egymás után, az utolsó az érvényes

def anomaly_handle_1(list):
    global anomaly_num
    anomaly_num = 1

    for j in range(len(list)):
        for i in range(len(list[j])-1):
            if(list[j]['SubStatus'].iloc[i] == 'ACTIVE' ):
                if(list[j]['SubStatus'].iloc[i+1] == 'ACTIVE'):     
                    list[j]['SubStatus'].iloc[i+1] = 'Anomaly'
            elif(list[j]['SubStatus'].iloc[i] == 'Anomaly'):
                if(list[j]['SubStatus'].iloc[i+1] == 'ACTIVE'):     
                    list[j]['SubStatus'].iloc[i+1] = 'Anomaly'
    for j in range(len(list)):
        for i in range(len(list[j])-1):
            if(list[j]['SubStatus'].iloc[i] == 'END' ):
                if(list[j]['SubStatus'].iloc[i+1] == 'END'):     
                    list[j]['SubStatus'].iloc[i] = 'Anomaly'

In [8]:
# Anomália kezelés 2: több ACTIVE jön egymás után, akkor az UTOLSÓ érvényes, több END jön egymás után, az utolsó az érvényes

def anomaly_handle_2(list):
    global anomaly_num
    anomaly_num = 2

    for j in range(len(list)):
        for i in range(len(list[j])-1):
            if(list[j]['SubStatus'].iloc[i] == 'ACTIVE' ):
                if(list[j]['SubStatus'].iloc[i+1] == 'ACTIVE'):     
                    list[j]['SubStatus'].iloc[i] = 'Anomaly'
    for j in range(len(list)):
        for i in range(len(list[j])-1):
            if(list[j]['SubStatus'].iloc[i] == 'END' ):
                if(list[j]['SubStatus'].iloc[i+1] == 'END'):     
                    list[j]['SubStatus'].iloc[i] = 'Anomaly'

In [9]:
anomaly_handle_2(list)

In [10]:
j = 0
k = 0
list_index = 0
for i in range(len(list[list_index])):
    if(list[list_index]['SubStatus'].iloc[i] == 'ACTIVE'):
        j = j + 1
    elif(list[list_index]['SubStatus'].iloc[i] == 'END'):
        k = k + 1

print("START: " + str(j))
print("STOP: " + str(k))

START: 1873
STOP: 1874


In [11]:
list_anomaly = []

for i in range(len(list)): 
    list_anomaly.insert(i, list[i]) 

In [12]:
for j in range(len(list)):
    list[j] = list[j][list[j]['SubStatus']!='Anomaly']

# idő hozzárendelése gépenként

In [13]:
# startal kell hogy kezdődjön és stoppal végződnie

In [14]:
list_anomaly[0]

,Id,Machine,StatusTime,SubStatus
272,776,10449,2019-08-21 14:10:57,END
381,1040,10449,2019-08-21 14:28:17,ACTIVE
554,1547,10449,2019-08-21 14:55:03,END
564,1595,10449,2019-08-21 14:57:01,ACTIVE
784,2365,10449,2019-08-21 15:37:15,END
...,...,...,...,...
613669,2008034,10449,2019-11-19 07:14:51,END
613675,2008048,10449,2019-11-19 07:15:31,ACTIVE
613766,2008247,10449,2019-11-19 07:28:53,END
613887,2008590,10449,2019-11-19 07:54:15,ACTIVE


In [15]:
for i in range(len(list_anomaly)):
    if(list_anomaly[i].iloc[0]['SubStatus'] != 'ACTIVE'):
        list_anomaly[i] = list_anomaly[i].drop([list_anomaly[i].index[0]])
    if(list_anomaly[i].iloc[len(list_anomaly[i]) - 1]['SubStatus'] != 'END'):
        list_anomaly[i] = list_anomaly[i].drop([list_anomaly[i].index[len(list_anomaly[i])-1]])

In [16]:
for i in range(len(list)):
    if(list[i].iloc[0]['SubStatus'] != 'ACTIVE'):
        list[i] = list[i].drop([list[i].index[0]])
    if(list[i].iloc[len(list[i]) - 1]['SubStatus'] != 'END'):
        list[i] = list[i].drop([list[i].index[len(list[i])-1]])

In [17]:
from datetime import datetime, timedelta

In [18]:
def timeInWorking(list): 
    for j in range(len(list)):    
        list[j]['timeInWorking'] = timedelta(0)
        for i in range(len(list[j])-1)[0::2]:
            timeACTIVE_Str = list[j]['StatusTime'].iloc[i]
            timeEND_Str = list[j]['StatusTime'].iloc[i+1]
            timeStart = datetime.strptime(timeACTIVE_Str,'%Y-%m-%d %H:%M:%S')
            timeEND = datetime.strptime(timeEND_Str,'%Y-%m-%d %H:%M:%S')
            list[j]['timeInWorking'].iloc[i+1] = timeEND-timeStart

In [19]:
def timeNotWorking(list): 
    for j in range(len(list)):
        list[j]['timeNotWorking'] = timedelta(0)
        for i in range(len(list[j])-1)[1::2]:
            timeACTIVE_Str = list[j]['StatusTime'].iloc[i]
            timeEND_Str = list[j]['StatusTime'].iloc[i+1]
            timeStart = datetime.strptime(timeACTIVE_Str,'%Y-%m-%d %H:%M:%S')
            timeEND = datetime.strptime(timeEND_Str,'%Y-%m-%d %H:%M:%S')
            list[j]['timeNotWorking'].iloc[i+1] = timeEND-timeStart

In [20]:
def timeInAnomaly(list):
    for j in range(len(list)):
        list[j]['timeInAnomaly'] = timedelta(0)

        i = 0
        while(i <= len(list[j])-1):
            if(list[j]['SubStatus'].iloc[i] == 'Anomaly' ):
                anomalyStart_Str = list[j]['StatusTime'].iloc[i]
                #ameddig anomália jön, addig megyünk
                k = 1
                while list[j]['SubStatus'].iloc[k + i] == 'Anomaly':
                    k = k + 1

                if(k == 1):
                    i += 1
                    pass
                else:
                    anomalyEND_Str = list[j]['StatusTime'].iloc[i + k]
                    timeStart = datetime.strptime(anomalyStart_Str,'%Y-%m-%d %H:%M:%S')
                    timeEND = datetime.strptime(anomalyEND_Str,'%Y-%m-%d %H:%M:%S')
                    list[j]['timeInAnomaly'].iloc[i + k] = timeEND-timeStart  
                    i = i + k
                    k = 1
            else:
                i += 1 

In [21]:
timeInAnomaly(list_anomaly)

In [22]:
timeInWorking(list)
timeNotWorking(list)

In [23]:
print('cycle in working time','','cycle when not working','','all time sum of the machine')
print('')
for i in range(len(list)):
    print(list[i]['timeInWorking'].sum(),'     ', list[i]['timeNotWorking'].sum(),'       ',list[i]['timeInWorking'].sum()+list[i]['timeNotWorking'].sum())

cycle in working time  cycle when not working  all time sum of the machine

46 days 21:00:31       42 days 21:43:09         89 days 18:43:40
59 days 18:42:57       30 days 03:45:03         89 days 22:28:00
40 days 11:22:57       49 days 12:20:52         89 days 23:43:49
33 days 20:39:11       19 days 18:42:39         53 days 15:21:50
62 days 20:14:11       27 days 03:33:22         89 days 23:47:33
61 days 03:40:36       28 days 15:52:51         89 days 19:33:27
50 days 06:58:14       39 days 12:51:36         89 days 19:49:50
50 days 01:16:44       39 days 21:25:00         89 days 22:41:44


In [24]:
for i in range(len(list_anomaly)):
    print(str(i+1) + " számú gép: ",list_anomaly[i]['timeInAnomaly'].sum())

1 számú gép:  2 days 17:35:47
2 számú gép:  15 days 15:56:57
3 számú gép:  2 days 17:56:54
4 számú gép:  2 days 12:10:12
5 számú gép:  21 days 23:48:24
6 számú gép:  12 days 01:14:57
7 számú gép:  10 days 03:15:27
8 számú gép:  1 days 22:54:34


In [25]:
# dolgok egybe gyúrása

In [26]:
for i in range(len(list_anomaly)-1):
    del list_anomaly[i]['Machine']
    del list_anomaly[i]['StatusTime']
    del list_anomaly[i]['SubStatus']

In [27]:
list_new = []
for i in range(len(list)-1):
    list_new.append(pd.merge(list[i], list_anomaly[i], left_on='Id', right_on='Id', how='left'))

In [28]:
df = pd.concat(list_new)

df.to_csv('cycle_status_3.csv', sep=";")

In [29]:
df[df['Machine']=='10704']

,Id,Machine,StatusTime,SubStatus,timeInWorking,timeNotWorking,timeInAnomaly
0,827,10704,2019-08-21 14:13:54,ACTIVE,0 days 00:00:00,00:00:00,0 days 00:00:00
1,848,10704,2019-08-21 14:14:53,END,0 days 00:00:59,00:00:00,0 days 00:00:00
2,903,10704,2019-08-21 14:18:47,ACTIVE,0 days 00:00:00,00:03:54,0 days 00:00:00
3,158543,10704,2019-08-28 19:15:37,END,7 days 04:56:50,00:00:00,7 days 04:38:11
4,160748,10704,2019-08-28 20:15:08,ACTIVE,0 days 00:00:00,00:59:31,0 days 00:00:00
...,...,...,...,...,...,...,...
4519,2005884,10704,2019-11-19 05:16:12,END,0 days 00:06:43,00:00:00,0 days 00:00:00
4520,2005907,10704,2019-11-19 05:18:33,ACTIVE,0 days 00:00:00,00:02:21,0 days 00:00:00
4521,2006147,10704,2019-11-19 05:25:33,END,0 days 00:07:00,00:00:00,0 days 00:00:00
4522,2006214,10704,2019-11-19 05:27:34,ACTIVE,0 days 00:00:00,00:02:01,0 days 00:00:00


In [30]:
import os
from datetime import datetime as dt
import datetime
import pytz, time
import gc

os.environ['TZ'] = 'Europe/Budpest'
CET = pytz.timezone('Europe/Budapest')

In [31]:
df = df[['Id', 'Machine', 'StatusTime', 'SubStatus', 'timeInWorking', 'timeNotWorking', 'timeInAnomaly']]
df = df.astype({'StatusTime' : 'datetime64'})
df.index = df.StatusTime
df.index = df.index.tz_localize('Europe/Budapest')
df = df.drop(columns='StatusTime')
df.sort_values(inplace=True, axis='index', by='StatusTime')
df.head()

,Id,Machine,SubStatus,timeInWorking,timeNotWorking,timeInAnomaly
StatusTime,,,,,,
2019-08-21 13:24:30+02:00,102,10395,ACTIVE,00:00:00,00:00:00,0 days
2019-08-21 13:25:30+02:00,137,10469,ACTIVE,00:00:00,00:00:00,0 days
2019-08-21 13:30:24+02:00,270,10395,END,00:05:54,00:00:00,0 days
2019-08-21 13:32:02+02:00,279,10395,ACTIVE,00:00:00,00:01:38,0 days
2019-08-21 13:36:18+02:00,357,10469,END,00:10:48,00:00:00,0 days


In [32]:
df['ts_work'] = 0
df['ts_notwork'] = 0
df['ts_anom'] = 0
for i in range(len(df)):
    df['ts_work'].iloc[i] = df['timeInWorking'].iloc[i].total_seconds() / 3600
    df['ts_notwork'].iloc[i] = df['timeNotWorking'].iloc[i].total_seconds() / 3600
    df['ts_anom'].iloc[i] = df['timeInAnomaly'].iloc[i].total_seconds() / 3600

In [33]:
del df['timeInWorking']
del df['timeNotWorking']
del df['timeInAnomaly']
df

,Id,Machine,SubStatus,ts_work,ts_notwork,ts_anom
StatusTime,,,,,,
2019-08-21 13:24:30+02:00,102,10395,ACTIVE,0.000000,0.000000,0.0
2019-08-21 13:25:30+02:00,137,10469,ACTIVE,0.000000,0.000000,0.0
2019-08-21 13:30:24+02:00,270,10395,END,0.098333,0.000000,0.0
2019-08-21 13:32:02+02:00,279,10395,ACTIVE,0.000000,0.027222,0.0
2019-08-21 13:36:18+02:00,357,10469,END,0.180000,0.000000,0.0
...,...,...,...,...,...,...
2019-11-19 12:48:35+01:00,2012454,10469,ACTIVE,0.000000,0.005556,0.0
2019-11-19 12:50:59+01:00,2012500,10395,END,0.217222,0.000000,0.0
2019-11-19 12:54:00+01:00,2012588,10395,ACTIVE,0.000000,0.050278,0.0


In [34]:
import influxdb

client = influxdb.DataFrameClient(host='152.66.34.82', port=8086)
client.switch_database('knorr')
client.query(query='DROP MEASUREMENT "cycle_status"', database='knorr')
client.write_points(df, database='knorr', measurement='cycle_status', tag_columns=['Machine', 'SubStatus'], field_columns=['ts_work','ts_notwork','ts_anom'], time_precision='ms', batch_size=10000, protocol='line')
client.query(query='SELECT * FROM cycle_status', database='knorr')

defaultdict(list,
            {'cycle_status':                           Machine SubStatus  ts_anom  ts_notwork   ts_work
             2019-08-21 11:24:30+00:00   10395    ACTIVE      0.0    0.000000  0.000000
             2019-08-21 11:25:30+00:00   10469    ACTIVE      0.0    0.000000  0.000000
             2019-08-21 11:30:24+00:00   10395       END      0.0    0.000000  0.098333
             2019-08-21 11:32:02+00:00   10395    ACTIVE      0.0    0.027222  0.000000
             2019-08-21 11:36:18+00:00   10469       END      0.0    0.000000  0.180000
             ...                           ...       ...      ...         ...       ...
             2019-11-19 11:48:35+00:00   10469    ACTIVE      0.0    0.005556  0.000000
             2019-11-19 11:50:59+00:00   10395       END      0.0    0.000000  0.217222
             2019-11-19 11:54:00+00:00   10395    ACTIVE      0.0    0.050278  0.000000
             2019-11-19 12:09:19+00:00   10469       END      0.0    0.000000  0.34555